In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise import KNNBaseline, SVDpp
from surprise.model_selection import GridSearchCV, cross_validate
from surprise import Dataset, Reader
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, RobustScaler, StandardScaler, MinMaxScaler

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error 
import math
import tensorflow.compat.v1 as tf


In [2]:
df = pd.read_csv('Data/Cleaned_Data/book_user_explicit_rating_cleaned.csv',encoding='UTF-8')
df = df.drop(columns=['index'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=['Image_URL','ISBN'])

In [3]:
labelenc = LabelEncoder()

In [4]:
def label_encoding(df,param):
    df[param] = labelenc.fit_transform(df[param].values)
    df[param] = df[param].astype('category')

In [5]:
label_encoding(df, 'User_ID')
label_encoding(df, 'Unique_ISBN')

In [6]:
enc_data = df[['User_ID','Unique_ISBN','Book_Rating']]

In [8]:
print ('num of users:',df['User_ID'].nunique())
print ('num of books:',df['Unique_ISBN'].nunique())

num of users: 20032
num of books: 116946


In [14]:
user_ids = np.array(df['User_ID'].tolist())
book_ids = np.array(df['Unique_ISBN'].tolist())
user_ratings = np.array(df["Book_Rating"].tolist())

In [12]:
graph = tf.Graph()
n_book = 116946
n_user = 20032
embedding_size = 30

lr = 0.0001
reg = 0.01

with graph.as_default():
    user = tf.placeholder(tf.int32, name="User_ID") 
    book = tf.placeholder(tf.int32, name="Unique_ISBN") 
    rating = tf.placeholder(tf.float32, name="Book_Rating") 

    book_embedding = tf.Variable(tf.truncated_normal([n_book, embedding_size], stddev=0.02, mean=0.) ,name="Book_Embedding")
    user_embedding = tf.Variable(tf.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="User_Embedding")
    
    book_bias_embedding = tf.Variable(tf.truncated_normal([n_book], stddev=0.02, mean=0.) ,name="book_bias_embedding")
    user_bias_embedding = tf.Variable(tf.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")
    
    
    global_bias = tf.Variable(tf.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")
    
    u = tf.nn.embedding_lookup(user_embedding, user)
    m = tf.nn.embedding_lookup(book_embedding, book)
    
    u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
    m_bias = tf.nn.embedding_lookup(book_bias_embedding, book)
    

    predicted_rating = tf.reduce_sum(tf.multiply(u, m), 1) + u_bias + m_bias + global_bias

    rmse = tf.sqrt(tf.reduce_mean(tf.square(predicted_rating - rating))) # RMSE
    cost = tf.nn.l2_loss(predicted_rating - rating)
    regularization = reg * (tf.nn.l2_loss(book_embedding) + tf.nn.l2_loss(user_embedding)
                            + tf.nn.l2_loss(book_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))
    
    loss = cost + regularization
    
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [15]:
batch_size = 5
n_epoch = 30


with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    for _ in range(n_epoch):
        for start in range(0, user_ratings.shape[0] - batch_size, batch_size):
            end = start + batch_size
            _, cost_value = sess.run([optimizer, rmse], feed_dict={user: user_ids[start:end],
                                                  book: book_ids[start: end],
                                                  rating: user_ratings[start: end]})

        print ("RMSE", cost_value)
    embeddings = book_embedding.eval()